In [47]:
import pandas as pd

df = pd.read_excel('../data/H1_analysis/has_cv_url.xlsx')
df

/Users/Paula_1/opt/anaconda3/envs/parenthood/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ResponseId,DE13c_1_TEXT,DE13a_Name,DE13_1_1,DE13_2_1,DE13_3_1,DE13_4_1,DE13_5_1,DE14_11_1,DE13_6_1
0,Response ID,We acknowledge your privacy considerations. Wo...,"As part of this study, we aim to gain insights...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli..."
1,R_77h3lBm8zYTS5a1,NaN,LisetteMunzCV.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R_4pD7OrlAt0l6mjL,NaN,NaN,https://scholar.google.com/citations?user=Guhy...,NaN,NaN,NaN,NaN,NaN,NaN
3,R_7OE9SfkJJ6q9jbP,NaN,NaN,https://scholar.google.com/citations?user=BSke...,NaN,NaN,NaN,NaN,NaN,NaN
4,R_11v7OTVCxgOhPWb,NaN,ORCID ID.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3231,R_2PV7RhvMAMP0hcR,NaN,CV_MWijga.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3232,R_2WwaSOfdPYRwAwN,NaN,NaN,NaN,0000-0003-1891-7522,NaN,NaN,NaN,NaN,NaN
3233,R_2w60Oo6hzaQimqC,NaN,NaN,https://scholar.google.co.uk/citations?user=Wi...,NaN,NaN,NaN,NaN,NaN,NaN
3234,R_2iFjsLGZX3qEgu0,NaN,NaN,https://scholar.google.com/citations?hl=en&use...,https://orcid.org/0000-0002-9203-7550,https://www.researchgate.net/profile/Elif-Calik,https://www.webofscience.com/wos/woscc/basic-s...,https://www.scopus.com/authid/detail.uri?autho...,NaN,NaN


In [48]:
df = df.drop(columns=["DE13c_1_TEXT"])

In [49]:
available_cvs = df['DE13a_Name'].notna().sum()
print(f"{available_cvs} CVs")

available_gscholar = df['DE13_1_1'].notna().sum()
print(f"{available_gscholar} Google Scholar accounts")

available_orcid = df['DE13_2_1'].notna().sum()
print(f"{available_orcid} Orcids")

available_rgate = df['DE13_3_1'].notna().sum()
print(f"{available_rgate} Research Gate accounts")

available_wscience = df['DE13_4_1'].notna().sum()
print(f"{available_wscience} Web of Science accounts")

available_scopus = df['DE13_5_1'].notna().sum()
print(f"{available_scopus} Scopus accounts")

available_oalex = df['DE14_11_1'].notna().sum()
print(f"{available_oalex} OpenAlex profiles found")

available_otherurl = df['DE13_6_1'].notna().sum()
print(f"{available_otherurl} other URLs found")

1178 CVs
929 Google Scholar accounts
1243 Orcids
402 Research Gate accounts
104 Web of Science accounts
181 Scopus accounts
8 OpenAlex profiles found
244 other URLs found


In [55]:
orcid_df = df.loc[df['DE13_2_1'].notna(), ['ResponseId', 'DE13_2_1']]
orcid_df

,ResponseId,DE13_2_1
0,Response ID,"Or if you prefer, you can also share your onli..."
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018
12,R_46yreKgPLBawn3H,0000-0002-2763-0921
13,R_2ZTGtufLu3KUHW9,0000000181844747
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496
...,...,...
3218,R_216eipKa8MqFn3z,https://orcid.org/0000-0002-6409-2632
3219,R_8FlyCLCDJMPasD4,https://orcid.org/0000-0002-6873-2967
3228,R_2X0fTWPBw20QquW,0000-0003-1528-8185
3232,R_2WwaSOfdPYRwAwN,0000-0003-1891-7522


In [63]:
failed_orcid_extractions = orcid_df[orcid_df['DE13_2_1'].isna()]
print(f"{len(failed_orcid_extractions)} failed ORCID extractions")

0 failed ORCID extractions


In [72]:
import re

def extract_canonical_orcid(value):
    value = str(value).strip()

    # Match dashed ORCID: 0000-0000-0000-0000 or ...-000X
    dashed = re.search(r'\d{4}-\d{4}-\d{4}-\d{3}[\dX]', value)
    if dashed:
        return f'https://orcid.org/{dashed.group(0)}'

    # Match raw 16-character ORCID: 15 digits + digit or X
    raw = re.search(r'\d{15}[\dX]', value)
    if raw:
        digits = raw.group(0)
        formatted = f'{digits[:4]}-{digits[4:8]}-{digits[8:12]}-{digits[12:]}'
        return f'https://orcid.org/{formatted}'

    # Everything else is ignored
    return None


In [73]:
orcid_df['orcid_standard'] = orcid_df['DE13_2_1'].apply(extract_canonical_orcid)
orcid_df

,ResponseId,DE13_2_1,orcid_standard
0,Response ID,"Or if you prefer, you can also share your onli...",None
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018,https://orcid.org/0000-0001-5041-5018
12,R_46yreKgPLBawn3H,0000-0002-2763-0921,https://orcid.org/0000-0002-2763-0921
13,R_2ZTGtufLu3KUHW9,0000000181844747,https://orcid.org/0000-0001-8184-4747
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496
...,...,...,...
3218,R_216eipKa8MqFn3z,https://orcid.org/0000-0002-6409-2632,https://orcid.org/0000-0002-6409-2632
3219,R_8FlyCLCDJMPasD4,https://orcid.org/0000-0002-6873-2967,https://orcid.org/0000-0002-6873-2967
3228,R_2X0fTWPBw20QquW,0000-0003-1528-8185,https://orcid.org/0000-0003-1528-8185
3232,R_2WwaSOfdPYRwAwN,0000-0003-1891-7522,https://orcid.org/0000-0003-1891-7522


In [74]:
failed_orcid_extractions = orcid_df[orcid_df['orcid_standard'].isna()]
print(f"{len(failed_orcid_extractions)} failed ORCID extractions")

18 failed ORCID extractions


In [75]:
# View the failed entries with original inputs
failed_orcid_extractions[['ResponseId', 'DE13_2_1']]


,ResponseId,DE13_2_1
0,Response ID,"Or if you prefer, you can also share your onli..."
29,R_4bHw34dQzUp3iPj,000000280613629
59,R_8YXmB75YGiuDUY4,2151684
1059,R_8kIvX5bxd8oEBuW,https://orcid.org/orcid-search/search?searchQu...
1068,R_2JKPy8nLPXl4GfT,0-0002-1525-5255
1234,R_2Ev6eGbhRDAGMUA,0000-0002-8988-81
1358,R_2KJgavGjIMAE5RD,We recommend that health advertisers:
1365,R_20tMEnRIJCDG0No,https://orcid.org/orcid-search/search?searchQu...
1436,R_8bIVRT6jaNhc7Ox,2
1705,R_6flVGEkboDnwB3E,0000-0002- 2652- 772X


In [71]:
# Show short or malformed non-matching entries (optional step)
suspicious_short = failed_orcid_extractions[failed_orcid_extractions['DE13_2_1'].str.len() < 15]
suspicious_short[['ResponseId', 'DE13_2_1']]


,ResponseId,DE13_2_1
59,R_8YXmB75YGiuDUY4,2151684
1436,R_8bIVRT6jaNhc7Ox,2
2385,R_8bK3iRDHBGuYulK,1 8091 9014
2887,R_81NJgXINQgcbhMt,00000022499
3047,R_8ery8MZOlUg82g1,0002-3523-8102
